In [7]:
import re
f = open('dataset/bing_liu/Nikon coolpix 4300.txt', 'r')

pattern_title = '\[t\]'
pattern_sentence = '(?<=##).+'
pattern_aspect = '.+(?=##)'
pattern_aspect_anomali = '\[(u|p|s|cc|cs)\]'

sentences = []
review = []
count = 0
for a in f:
    if re.search('##', a):
        sentence = re.findall(pattern_sentence, a)[0]
        aspect = re.findall(pattern_aspect, a)
#         anomali = re.findall(pattern_aspect_anomali, a)
#         if anomali:
#             print(sentence, aspect)
#         else:
        if len(aspect) > 0:
            aspect = aspect[0]
        else:
            aspect = ''
        sentences.append((sentence, aspect))

In [8]:
import pandas as pd

df = pd.DataFrame(columns=['review','aspect'])
for r in sentences:
    df = df.append({'review': r[0], 'aspect': r[1]}, ignore_index=True)
df.to_csv("nikon.csv")

In [2]:
f = open("SentiCircle/nokia.txt","w+")
for i in sentences:
     f.write(i + "\n")
f.close()

In [11]:
import pandas as pd
import xml.etree.ElementTree as et 
def parse_laptop(filename, isMultipleCategories=False):
    df_cols = ["reviewID", "sentenceID", "review", "category", "polarity"]
    out_df = pd.DataFrame(columns = df_cols)

    xtree_train = et.parse(filename)
    xroot_train = xtree_train.getroot()

    for node in xroot_train: 
        rid = node.attrib.get('rid')
        sentences = node.find('sentences') if node is not None else None
        for sentence in sentences:
            sid = sentence.attrib.get('id')
            text = sentence.find('text').text if sentence is not None else None
            opinions = sentence.find('Opinions') if sentence is not None else []
            if opinions is not None:
                categories = []
                polarities = []
                for opinion in opinions:
                    categories.append(opinion.attrib.get('category'))
                    polarities.append(opinion.attrib.get('polarity'))
                if not isMultipleCategories:
                    out_df = out_df.append(pd.Series([rid, sid, text, categories[0], polarities[0]], 
                                        index = df_cols), 
                            ignore_index = True)
                else:
                    out_df = out_df.append(pd.Series([rid, sid, text, ','.join(categories), ','.join(polarities)], 
                                        index = df_cols), 
                            ignore_index = True)

    return out_df

In [13]:
res_df_laptop_single = parse_laptop("dataset/ABSA16/ABSA16_Laptops_Train_SB1_v2.xml")
res_df_laptop_multiple = parse_laptop("dataset/ABSA16/ABSA16_Laptops_Train_SB1_v2.xml", True)

res_df_laptop_single.to_csv('dataset/laptop-single-train.csv', index=False)
res_df_laptop_multiple.to_csv('dataset/laptop-multiple-train.csv', index=False)

In [14]:
res_df_laptop_single = parse_laptop("dataset/ABSA15_RestaurantsTrain/ABSA-15_Restaurants_Train_Final.xml")
res_df_laptop_multiple = parse_laptop("dataset/ABSA15_RestaurantsTrain/ABSA-15_Restaurants_Train_Final.xml", True)

res_df_laptop_single.to_csv('dataset/restaurant-single-test.csv', index=False)
res_df_laptop_multiple.to_csv('dataset/restaurant-multiple-test.csv', index=False)